In [7]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

## Notebook for Inflation Analysis
* Dhruv Pandit 2022

In [8]:
df_inflation = pd.read_excel(r'/Users/dhruvpandit/Documents/GitHub/time-series-project/Datasets/custom_transpose_dataset.xlsx', sheet_name='Sheet transpose')

In [9]:
df_inflation

,TIME,"European Union (EU6-1958, EU9-1973, EU10-1981, EU12-1986, EU15-1995, EU25-2004, EU27-2007, EU28-2013, EU27-2020)",European Union - 27 countries (from 2020),European Union - 28 countries (2013-2020),"Euro area (EA11-1999, EA12-2001, EA13-2007, EA15-2008, EA16-2009, EA17-2011, EA18-2014, EA19-2015)",Euro area - 19 countries (from 2015),Euro area - 18 countries (2014),Belgium,Bulgaria,Czechia,...,Finland,Sweden,"European Economic Area (EEA18-1995, EEA28-2004, EEA30-2007, EEA31-2013, EEA30-2020)",Iceland,Norway,Switzerland,United Kingdom,North Macedonia,Serbia,Türkiye
0,1996-01,69.4,:,:,70.97,70.4,70.46,69.91,:,57,...,71.2,75.41,69.37,42.75,69.2,:,67.8,:,:,1.2
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,d
2,1996-02,69.7,:,:,71.29,70.71,70.76,69.98,:,57.3,...,71.49,75.6,69.67,42.83,69.3,:,68.1,:,:,1.25
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,d
4,1996-03,70,:,:,71.54,71.01,71.06,70.12,:,57.6,...,71.63,76.14,69.97,42.92,69.5,:,68.4,:,:,1.32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
639,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,d,d,d
640,2022-09,122.07,121.56,:,119.26,119.26,119.13,125.9,127.1,136.8,...,115.2,122.7,122.14,114.62,128,104.53,:,132.18,132.7,401.1
641,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,d,d,d
642,2022-10,:,:,:,121.07,:,:,129.3,:,:,...,116.05,:,:,:,:,:,:,:,:,:


In [10]:
df_inflation.dropna(axis=0, inplace=True)
df_inflation.reset_index()

,index,TIME,"European Union (EU6-1958, EU9-1973, EU10-1981, EU12-1986, EU15-1995, EU25-2004, EU27-2007, EU28-2013, EU27-2020)",European Union - 27 countries (from 2020),European Union - 28 countries (2013-2020),"Euro area (EA11-1999, EA12-2001, EA13-2007, EA15-2008, EA16-2009, EA17-2011, EA18-2014, EA19-2015)",Euro area - 19 countries (from 2015),Euro area - 18 countries (2014),Belgium,Bulgaria,...,Finland,Sweden,"European Economic Area (EEA18-1995, EEA28-2004, EEA30-2007, EEA31-2013, EEA30-2020)",Iceland,Norway,Switzerland,United Kingdom,North Macedonia,Serbia,Türkiye
0,0,1996-01,69.4,:,:,70.97,70.4,70.46,69.91,:,...,71.2,75.41,69.37,42.75,69.2,:,67.8,:,:,1.2
1,2,1996-02,69.7,:,:,71.29,70.71,70.76,69.98,:,...,71.49,75.6,69.67,42.83,69.3,:,68.1,:,:,1.25
2,4,1996-03,70,:,:,71.54,71.01,71.06,70.12,:,...,71.63,76.14,69.97,42.92,69.5,:,68.4,:,:,1.32
3,6,1996-04,70.19,:,:,71.66,71.14,71.19,70.47,:,...,71.78,76.49,70.15,43.09,69.7,:,68.7,:,:,1.42
4,8,1996-05,70.37,:,:,71.83,71.32,71.36,70.61,:,...,71.99,76.56,70.34,43.27,69.9,:,68.9,:,:,1.49
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
317,634,2022-06,119.53,119.03,:,117.01,117.01,116.9,122.82,124.16,...,114.31,119.61,119.59,113.82,124.3,104.27,:,126.25,127.9,374.67
318,636,2022-07,119.81,119.31,:,117.14,117.14,117.03,122.03,125.18,...,114.49,119.36,119.88,115.16,126,104.42,:,127.71,129.2,383.5
319,638,2022-08,120.61,120.1,:,117.85,117.85,117.74,124.24,126.2,...,114.37,121.22,120.67,114.89,125.7,104.72,:,130.05,130.7,389.05
320,640,2022-09,122.07,121.56,:,119.26,119.26,119.13,125.9,127.1,...,115.2,122.7,122.14,114.62,128,104.53,:,132.18,132.7,401.1


In [11]:
df_inflation['TIME']

0      1996-01
2      1996-02
4      1996-03
6      1996-04
8      1996-05
        ...   
634    2022-06
636    2022-07
638    2022-08
640    2022-09
642    2022-10
Name: TIME, Length: 322, dtype: object